In [1]:
pip install insightface onnxruntime opencv-python pillow tqdm numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071488 sha256=a7cdc3ab969e560f7c1ab33d573310291904d825be96cfa2511539daa880a9df
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [2]:
!pip install beautifulsoup4

In [3]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

KOBIS_API_KEY = os.getenv("KOBIS_API_KEY")

API_MOVIE_LIST = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"
API_MOVIE_INFO = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"
API_PEOPLE_LIST = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"

PEOPLE_PAGE = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (compatible; KOBIS-crawler/1.0)"
})

In [4]:
import re
from datetime import datetime

def _norm(s: str) -> str:
    """비교용 정규화: 공백/특수문자 제거 + 소문자"""
    s = (s or "").strip()
    s = re.sub(r"\s+", "", s)
    s = re.sub(r"[^\w가-힣]", "", s)
    return s.lower()

def _parse_date_yyyymmdd(s: str) -> datetime | None:
    s = (s or "").strip()
    if not s:
        return None
    try:
        return datetime.strptime(s, "%Y-%m-%d")
    except ValueError:
        # openDt가 'YYYYMMDD'로 오는 경우 대비
        try:
            return datetime.strptime(s, "%Y%m%d")
        except ValueError:
            return None

def get_movie_cd(movie_name: str, director_name: str | None = None) -> str | None:
    """영화명 + (선택) 감독명으로 movieCd(영화코드) 가져오기"""

    params = {
        "key": KOBIS_API_KEY,
        "movieNm": movie_name,
        "itemPerPage": 100,  # 후보 많이 받아서 후처리하기
    }
    if director_name:
        params["directorNm"] = director_name

    r = session.get(API_MOVIE_LIST, params=params)
    r.raise_for_status()
    data = r.json()
    movies = data.get("movieListResult", {}).get("movieList", [])
    if not movies:
        return None

    # 2) 2차: 결과가 여러 개면 “제목 정확도 + 감독 매칭 + 최신 개봉일”로 스코어링
    q_title = _norm(movie_name)
    q_dir = _norm(director_name) if director_name else ""

    best = None
    best_score = -1
    best_open_dt = None

    for m in movies:
        title = _norm(m.get("movieNm", ""))
        score = 0

        # 제목: 완전일치 > 부분일치
        if title == q_title:
            score += 10
        elif q_title and q_title in title:
            score += 6

        # 감독: movieList 응답에 directors가 오는 경우가 많음(없으면 빈 처리)
        if q_dir:
            directors = m.get("directors") or []
            dir_names = " ".join([d.get("peopleNm", "") for d in directors])
            if q_dir and q_dir in _norm(dir_names):
                score += 10

        # 동률이면 최신 개봉일(openDt) 우선
        open_dt = _parse_date_yyyymmdd(m.get("openDt", ""))

        if (score > best_score) or (score == best_score and (open_dt or datetime.min) > (best_open_dt or datetime.min)):
            best = m
            best_score = score
            best_open_dt = open_dt

    return best["movieCd"] if best else movies[0]["movieCd"]

def get_main_actors(movie_cd: str, max_actors: int = 10) -> list[dict]:
    """movieCd로 영화 상세에서 배우 목록 중 상위 max_actors명을 '주연'처럼 사용"""
    params = {"key": KOBIS_API_KEY, "movieCd": movie_cd}
    r = session.get(API_MOVIE_INFO, params=params)
    r.raise_for_status()
    data = r.json()

    movie_info = data["movieInfoResult"]["movieInfo"]
    actors = movie_info.get("actors", [])

    # KOBIS API는 '주연/조연' 구분이 없으므로, 상위 N명을 대표 배우로 사용
    return actors[:max_actors]


def get_people_cd(actor_name: str, movie_name: str, prefer_role: str = "배우") -> str | None:
    params = {
        "key": KOBIS_API_KEY,
        "peopleNm": actor_name,
        "filmoNames": movie_name,
        "itemPerPage": 10,
    }
    r = session.get(API_PEOPLE_LIST, params=params)
    r.raise_for_status()
    data = r.json()

    people_list = data.get("peopleListResult", {}).get("peopleList", [])
    if not people_list:
        return None

    # 1순위: 영화 제목이 filmoNames에 포함되고, 대표분야가 '배우' 인 사람
    candidates = [
        p for p in people_list
        if movie_name in (p.get("filmoNames") or "")
        and p.get("repRoleNm") == prefer_role
    ]

    # 2순위: 대표분야만 '배우' 인 사람
    if not candidates:
        candidates = [
            p for p in people_list
            if p.get("repRoleNm") == prefer_role
        ]

    # else => 그냥 첫 번째
    if not candidates:
        candidates = people_list

    chosen = candidates[0]
    """
    print(
        f"[get_people_cd] 선택: {chosen.get('peopleNm')} / "
        f"{chosen.get('repRoleNm')} / peopleCd={chosen.get('peopleCd')}"
    )
    """
    return chosen["peopleCd"]

def get_actor_images_from_people_page(people_cd: str) -> list[str]:
    """
    영화인 코드(peopleCd)로 KOBIS 모바일 영화인 상세 페이지를 가져와서
    사람 사진(/common/mast/people/) URL들을 추출.
    """
    DETAIL_URL = "https://www.kobis.or.kr/kobis/mobile/mast/peop/searchPeopleDtl.do"

    params = {"peopleCd": people_cd}
    r = session.get(DETAIL_URL, params=params)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    image_urls: list[str] = []

    # 페이지 안의 모든 img 중 사람 사진 경로만 수집
    for img in soup.find_all("img"):
        src = img.get("src") or ""
        if "/common/mast/people/" not in src:
            continue
        full = urljoin(r.url, src)
        if full not in image_urls:
            image_urls.append(full)

    return image_urls

def get_main_actor_images(movie_name: str) -> dict[str, dict[str, str]]:
    """
    영화명 → 배우 → (배역 이름 + 대표 이미지 1장 URL)
    리턴 예시:
    {
        "송강호": {
            "cast": "김기택",
            "image_url": "https://....jpg"
        },
        ...
    }
    """
    movie_cd = get_movie_cd(movie_name)
    if not movie_cd:
        raise ValueError(f"영화명을 찾을 수 없음: {movie_name}")

    # KOBIS movieInfo API에서 배우 목록 가져오기
    main_actors = get_main_actors(movie_cd)
    result: dict[str, dict[str, str]] = {}

    for actor in main_actors:
        name = actor["peopleNm"]               # 배우 이름
        cast = actor.get("cast", "") or ""     # 등장인물 이름(배역명), 없으면 빈 문자열

        print(f"[+] 주연 배우 처리 중: {name} / 배역: {cast}")

        # 배우 + 영화 제목으로 peopleCd 찾기
        people_cd = get_people_cd(name, movie_name)
        if not people_cd:
            print(f"    -> peopleCd를 찾지 못했습니다.")
            continue

        # 영화인 상세 페이지에서 인물 사진 URL 리스트 추출
        img_urls = get_actor_images_from_people_page(people_cd)
        if not img_urls:
            print(f"    -> 배우 사진 URL을 찾지 못했습니다.")
            continue

        # 첫 번째 이미지를 대표 사진으로 사용
        result[name] = {
            "cast": cast,
            "image_url": img_urls[0],
        }

    return result

In [5]:
movie_title = "기생충"
director = "봉준호"

movie_cd = get_movie_cd(movie_title, director_name=director)
print("movieCd:", movie_cd)

images_by_actor = get_main_actor_images(movie_title)
print(images_by_actor)

movieCd: 20183782
[+] 주연 배우 처리 중: 송강호 / 배역: 기택
[+] 주연 배우 처리 중: 이선균 / 배역: 동익
[+] 주연 배우 처리 중: 조여정 / 배역: 연교
[+] 주연 배우 처리 중: 최우식 / 배역: 기우
[+] 주연 배우 처리 중: 박소담 / 배역: 기정
[+] 주연 배우 처리 중: 이정은 / 배역: 문광
[+] 주연 배우 처리 중: 장혜진 / 배역: 충숙
[+] 주연 배우 처리 중: 박명훈 / 배역: 근세
[+] 주연 배우 처리 중: 정지소 / 배역: 다혜
[+] 주연 배우 처리 중: 정현준 / 배역: 다송
{'송강호': {'cast': '기택', 'image_url': 'https://www.kobis.or.kr/common/mast/people/2017/07/thumb_x110/thn_212ae19d375049fa86ffa72bed520a28.jpg'}, '이선균': {'cast': '동익', 'image_url': 'https://www.kobis.or.kr/common/mast/people/2019/09/thumb_x110/thn_0891637b67014d9a9d6b47556c2ad355.jpg'}, '조여정': {'cast': '연교', 'image_url': 'https://www.kobis.or.kr/common/mast/people/2020/02/thumb_x110/thn_352368f0358b4932974e3578330679a6.jpg'}, '최우식': {'cast': '기우', 'image_url': 'https://www.kobis.or.kr/common/mast/people/2020/01/thumb_x110/thn_1f768834f6834c75b9249328e69a18a9.jpg'}, '박소담': {'cast': '기정', 'image_url': 'https://www.kobis.or.kr/common/mast/people/2019/10/thumb_x110/thn_489ac59594a84cbf8f6d7

In [6]:
!apt-get -y update
!apt-get -y install fonts-noto-cjk

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,233 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,586 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRele

In [7]:
import os

os.makedirs("actor_images", exist_ok=True)

movie_title = "기생충"
actors_info = get_main_actor_images(movie_title)
# actors_info: {배우이름: {"cast": 배역명, "image_url": ...}}

for actor, info in actors_info.items():
    cast = info.get("cast", "")
    url = info["image_url"]

    resp = session.get(url)
    resp.raise_for_status()

    ext = os.path.splitext(url)[1] or ".jpg"

    # 파일 = 배우 이름 + 배역명 / 예: 송강호_김기택.jpg
    safe_actor = actor.replace(" ", "_")
    safe_cast = cast.replace(" ", "_") if cast else ""
    if safe_cast:
        filename = f"actor_images/{safe_actor}_{safe_cast}{ext}"
    else:
        filename = f"actor_images/{safe_actor}{ext}"

    with open(filename, "wb") as f:
        f.write(resp.content)

    print(f"saved: {filename} (배우: {actor}, 배역: {cast})")

[+] 주연 배우 처리 중: 송강호 / 배역: 기택
[+] 주연 배우 처리 중: 이선균 / 배역: 동익
[+] 주연 배우 처리 중: 조여정 / 배역: 연교
[+] 주연 배우 처리 중: 최우식 / 배역: 기우
[+] 주연 배우 처리 중: 박소담 / 배역: 기정
[+] 주연 배우 처리 중: 이정은 / 배역: 문광
[+] 주연 배우 처리 중: 장혜진 / 배역: 충숙
[+] 주연 배우 처리 중: 박명훈 / 배역: 근세
[+] 주연 배우 처리 중: 정지소 / 배역: 다혜
[+] 주연 배우 처리 중: 정현준 / 배역: 다송
saved: actor_images/송강호_기택.jpg (배우: 송강호, 배역: 기택)
saved: actor_images/이선균_동익.jpg (배우: 이선균, 배역: 동익)
saved: actor_images/조여정_연교.jpg (배우: 조여정, 배역: 연교)
saved: actor_images/최우식_기우.jpg (배우: 최우식, 배역: 기우)
saved: actor_images/박소담_기정.jpg (배우: 박소담, 배역: 기정)
saved: actor_images/이정은_문광.jpg (배우: 이정은, 배역: 문광)
saved: actor_images/장혜진_충숙.jpg (배우: 장혜진, 배역: 충숙)
saved: actor_images/박명훈_근세.jpg (배우: 박명훈, 배역: 근세)
saved: actor_images/정지소_다혜.jpg (배우: 정지소, 배역: 다혜)
saved: actor_images/정현준_다송.jpg (배우: 정현준, 배역: 다송)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
import json
import math
from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
from insightface.app import FaceAnalysis


# 유틸: 갤러리 파일명에서 (actor, cast) 파싱
def parse_actor_cast_from_filename(p: Path):
    stem = p.stem.strip()

    # 우선순위: "__" > "_" > "-" > 공백
    for sep in ["__", "_", "-", " "]:
        if sep in stem:
            a, b = stem.split(sep, 1)
            return a.strip(), b.strip()

    return stem, ""


# 갤러리 임베딩 생성
def get_first_face_embedding(img_bgr: np.ndarray, app: FaceAnalysis):
    faces = app.get(img_bgr)
    if not faces:
        return None
    # 가장 큰 얼굴을 쓰고 싶으면 bbox area 기준 정렬 추천
    faces = sorted(faces, key=lambda f: (f.bbox[2]-f.bbox[0])*(f.bbox[3]-f.bbox[1]), reverse=True)
    emb = faces[0].normed_embedding.astype(np.float32)  # 이미 normed인 경우가 많음
    emb = emb / (np.linalg.norm(emb) + 1e-12)
    return emb


def build_gallery(ACTOR_DIR: Path, app: FaceAnalysis):
    actor_labels = []
    cast_labels = []
    embs = []

    img_exts = {".jpg", ".jpeg", ".png", ".webp"}

    files = [p for p in sorted(ACTOR_DIR.glob("*")) if p.suffix.lower() in img_exts]
    if not files:
        raise FileNotFoundError(f"갤러리 이미지가 없습니다: {ACTOR_DIR}")

    for p in tqdm(files, desc="Building gallery"):
        img = cv2.imread(str(p))
        if img is None:
            continue

        emb = get_first_face_embedding(img, app)
        if emb is None:
            # 얼굴이 안 잡힌 이미지는 스킵
            continue

        actor, cast = parse_actor_cast_from_filename(p)
        actor_labels.append(actor)
        cast_labels.append(cast)
        embs.append(emb)

    if not embs:
        raise RuntimeError("갤러리에서 얼굴 임베딩을 하나도 만들지 못했습니다. 갤러리 이미지/품질을 확인하세요.")

    gallery_embeddings = np.vstack(embs).astype(np.float32)  # (N, D)
    return gallery_embeddings, actor_labels, cast_labels


def match_embedding_to_gallery(face_emb: np.ndarray,
                               gallery_embeddings: np.ndarray,
                               gallery_actor_labels,
                               gallery_cast_labels,
                               sim_threshold: float = 0.2):
    # cosine similarity: (N,D) @ (D,)  (둘다 L2 normalize 되어있다고 가정)
    sims = gallery_embeddings @ face_emb
    best_idx = int(np.argmax(sims))
    best_sim = float(sims[best_idx])

    if best_sim < sim_threshold:
        return "Unknown", "", best_sim

    return gallery_actor_labels[best_idx], gallery_cast_labels[best_idx], best_sim



# 한글 텍스트 렌더링용 폰트 로드
def load_korean_font(font_size=22):
    candidates = [
        "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc",
        "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc",
        "/usr/share/fonts/opentype/noto/NotoSansCJKkr-Regular.otf",
    ]
    for fp in candidates:
        if os.path.exists(fp):
            return ImageFont.truetype(fp, font_size)
    return ImageFont.load_default()


# 비디오 처리 (라벨링 비디오 저장 + JSONL 저장)
def label_video(
    video_path: Path,
    out_video_path: Path,
    out_jsonl_path: Path | None,
    app: FaceAnalysis,
    gallery_embeddings: np.ndarray,
    gallery_actor_labels,
    gallery_cast_labels,
    sim_threshold: float = 0.2,
    process_every_n_frames: int = 1,
    resize_width: int | None = 960,
    hold_frames: int | None = None,
):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise FileNotFoundError(f"비디오를 열 수 없습니다: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 0

    if hold_frames is None:
        hold_frames = process_every_n_frames

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(out_video_path), fourcc, fps, (w, h))

    font = load_korean_font(22)

    cached = []
    cached_at = -10**9

    frame_idx = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        do_detect = (frame_idx % process_every_n_frames) == 0

        if do_detect:
            scale = 1.0
            det_frame = frame
            if resize_width is not None and w > resize_width:
                scale = resize_width / float(w)
                new_h = int(h * scale)
                det_frame = cv2.resize(frame, (resize_width, new_h), interpolation=cv2.INTER_AREA)

            faces = app.get(det_frame)

            new_cached = []
            for f in faces:
                emb = f.normed_embedding.astype(np.float32)
                emb = emb / (np.linalg.norm(emb) + 1e-12)

                bbox = f.bbox.astype(np.float32)
                if scale != 1.0:
                    bbox = bbox / scale
                x1, y1, x2, y2 = map(int, bbox.tolist())

                actor, cast, sim = match_embedding_to_gallery(
                    emb, gallery_embeddings, gallery_actor_labels, gallery_cast_labels, sim_threshold
                )

                label = f"{actor}"
                if cast:
                    label += f" ({cast})"
                label += f"  {sim:.2f}"

                new_cached.append((x1, y1, x2, y2, label))

            if new_cached:
                cached = new_cached
                cached_at = frame_idx

        if (frame_idx - cached_at) > hold_frames:
            cached = []

        # 여기서부터는 매 프레임 캐시를 그려서 저장
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil = Image.fromarray(img_rgb)
        draw = ImageDraw.Draw(pil)

        for (x1, y1, x2, y2, label) in cached:
            draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=3)
            tw, th = draw.textbbox((0, 0), label, font=font)[2:]
            draw.rectangle([x1, max(0, y1 - th - 8), x1 + tw + 8, y1], fill=(0, 0, 0))
            draw.text((x1 + 4, max(0, y1 - th - 4)), label, font=font, fill=(255, 255, 255))

        out_bgr = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
        writer.write(out_bgr)

        frame_idx += 1

    cap.release()
    writer.release()

In [12]:
from pathlib import Path
import re


CLIPS_DIR = Path("/content/drive/MyDrive/Tave 16기 심화프로젝트/TAVE 16th/data/split_scenes/clips_2")

# 갤러리 폴더(배우 대표 얼굴 이미지들)
ACTOR_DIR = Path("/content/actor_images")

# 출력 폴더(라벨링된 mp4들)
OUT_DIR = Path("/content/drive/MyDrive/Tave 16기 심화프로젝트/TAVE 16th/data/split_scenes/clips_2_faceLabeled2")
OUT_DIR.mkdir(parents=True, exist_ok=True)


# InsightFace 준비
from insightface.app import FaceAnalysis

app = FaceAnalysis(
    name="buffalo_l",
    root="./insightface_model_cache",
    allowed_modules=["detection", "recognition"],
)
app.prepare(ctx_id=0, det_size=(640, 640))

# 갤러리 구축 (한 번만)
gallery_embeddings, gallery_actor_labels, gallery_cast_labels = build_gallery(ACTOR_DIR, app)

# 입력 비디오 목록 수집
video_paths = sorted(CLIPS_DIR.glob("*.mp4"))
print(f"Found {len(video_paths)} mp4 files in: {CLIPS_DIR}")

if not video_paths:
    raise FileNotFoundError(f"mp4 파일이 없습니다: {CLIPS_DIR}")


# 배치 처리
#   - 출력: mp4만 저장
#   - JSONL 저장 안 함(out_jsonl_path=None)
def sort_key(p: Path):
    nums = re.findall(r"\d+", p.stem)
    return (int(nums[0]) if nums else 10**18, p.name.lower())

video_paths = sorted(video_paths, key=sort_key)

start, end = 61, 62
batch_paths = video_paths[start:end]

failed = []
for vp in tqdm(batch_paths, desc=f"Batch labeling"):
    try:
        out_video = OUT_DIR / f"{vp.stem}_labeled.mp4"

        # 이미 생성된 파일이면 스킵(중복 실행 방지)
        if out_video.exists():
            continue

        info = label_video(
            video_path=vp,
            out_video_path=out_video,
            out_jsonl_path=None,
            app=app,
            gallery_embeddings=gallery_embeddings,
            gallery_actor_labels=gallery_actor_labels,
            gallery_cast_labels=gallery_cast_labels,
            sim_threshold=0.2,
            process_every_n_frames=10,
            resize_width=960,
        )
    except Exception as e:
        failed.append((str(vp), repr(e)))

print(f"Done. success={len(batch_paths)-len(failed)}, failed={len(failed)}")
if failed:
    print("---- failed list ----")
    for f in failed:
        print(f)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: ./insightface_model_cache/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: ./insightface_model_cache/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./insightface_model_cache/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: ./insightface_model_cache/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./insightface_model_cache/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


Building gallery: 100%|██████████| 10/10 [00:08<00:00,  1.22it/s]


Found 988 mp4 files in: /content/drive/MyDrive/Tave 16기 심화프로젝트/TAVE 16th/data/split_scenes/clips_2


Batch labeling: 100%|██████████| 1/1 [00:00<00:00, 456.10it/s]

Done. success=1, failed=0
